In [1]:
# Importing Some of the most important modules
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.models import Sequential
from keras import layers
import os
import tensorflow as tf
import pandas as pd

In [2]:
#Importing data 
#Data can ber ob""
df = pd.read_csv('data.csv')


,Text,DepType
0,Why did she leave me,3
1,My parents dont respect my freedom,2
2,Why am I so lonely,4
3,Why do i have all this work,8
4,I cant sleep,0
...,...,...
69,I hate celebrities so much,5
70,How come football players make so much money,9
71,I dont want to do a single thing,8
72,I have so much work and yet i dont want to do ...,8


In [3]:
from sklearn.model_selection import train_test_split
text = df['Text'].values
deptype=df['DepType'].values
text_train,text_test,deptype_train,deptype_test = train_test_split(
text,deptype, test_size=0.25, random_state=1000) 
deptype_train=np.asarray(deptype_train).astype('float32').reshape((-1,1))
deptype_test=np.asarray(deptype_test).astype('float32').reshape((-1,1))


In [4]:
 from sklearn.feature_extraction.text import CountVectorizer
 review_vectorizer = CountVectorizer()
 review_vectorizer.fit(text_train)
 Xlr_train = review_vectorizer.transform(text_train)
 Xlr_test  = review_vectorizer.transform(text_test)

In [5]:
 from sklearn.linear_model import LogisticRegression
 LRmodel = LogisticRegression()
 LRmodel.fit(Xlr_train, deptype_train)
 score = LRmodel.score(Xlr_test, deptype_test)
 print("Accuracy:", score)

Accuracy: 0.5789473684210527


C:\Users\ALIFF\anaconda3\envs\ML Project\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(text_train)
Xcnn_train = tokenizer.texts_to_sequences(text_train)
Xcnn_test = tokenizer.texts_to_sequences(text_test)
vocab_size = len(tokenizer.word_index) + 1  
print(text_train[1])
print(Xcnn_train[1]) 

School is stupid and i hate everyone in it
[17, 13, 14, 31, 1, 5, 32, 33, 64]


In [7]:
from tensorflow import keras
pad_sequences=keras.preprocessing.sequence.pad_sequences
maxlen = 100
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
# print(Xcnn_train[0, :])

In [8]:
 from keras.models import Sequential
 from keras import layers 
def create_model():
    embedding_dim = 200
    textcnnmodel = Sequential()
    textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    textcnnmodel.add(layers.Conv1D(128, 9, activation='relu'))
    textcnnmodel.add(layers.GlobalMaxPooling1D())
    textcnnmodel.add(layers.Dense(10, activation='relu'))
    textcnnmodel.add(layers.Dense(10, activation='softmax'))
    textcnnmodel.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
    textcnnmodel.summary() 
    return textcnnmodel
textcnnmodel=create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 200)          31000     
                                                                 
 conv1d (Conv1D)             (None, 92, 128)           230528    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
Total params: 262,928
Trainable params: 262,928
Non-trainable params: 0
__________________________________________________

In [9]:
import os
import tensorflow as tf
checkpoint_path = "training_2/1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size = 32
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

textcnnmodel.save_weights(checkpoint_path.format(epoch=0))


In [73]:
textcnnmodel.fit(Xcnn_train, deptype_train,
                 epochs=1000,
                 verbose=2,
                 validation_data=(Xcnn_test, deptype_test),
                callbacks=[cp_callback],

                 batch_size=10)
loss, accuracy = textcnnmodel.evaluate(Xcnn_train, deptype_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn_test, deptype_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy)) 

Epoch 1/1000
6/6 - 0s - loss: 0.2728 - accuracy: 0.9636 - val_loss: 2.5126 - val_accuracy: 0.4737 - 163ms/epoch - 27ms/step
Epoch 2/1000
6/6 - 0s - loss: 0.2175 - accuracy: 1.0000 - val_loss: 2.6076 - val_accuracy: 0.4737 - 144ms/epoch - 24ms/step
Epoch 3/1000
6/6 - 0s - loss: 0.1768 - accuracy: 1.0000 - val_loss: 2.6693 - val_accuracy: 0.4737 - 130ms/epoch - 22ms/step
Epoch 4/1000
6/6 - 0s - loss: 0.1379 - accuracy: 1.0000 - val_loss: 2.6423 - val_accuracy: 0.4737 - 123ms/epoch - 20ms/step
Epoch 5/1000
6/6 - 0s - loss: 0.1067 - accuracy: 1.0000 - val_loss: 2.7749 - val_accuracy: 0.4737 - 119ms/epoch - 20ms/step
Epoch 6/1000
6/6 - 0s - loss: 0.0860 - accuracy: 1.0000 - val_loss: 2.8394 - val_accuracy: 0.4737 - 135ms/epoch - 22ms/step
Epoch 7/1000

Epoch 7: saving model to training_2\1.ckpt
6/6 - 0s - loss: 0.0691 - accuracy: 1.0000 - val_loss: 2.8525 - val_accuracy: 0.4737 - 193ms/epoch - 32ms/step
Epoch 8/1000
6/6 - 0s - loss: 0.0569 - accuracy: 1.0000 - val_loss: 2.9682 - val_accurac

Epoch 66/1000
6/6 - 0s - loss: 0.0019 - accuracy: 1.0000 - val_loss: 4.2280 - val_accuracy: 0.4737 - 192ms/epoch - 32ms/step
Epoch 67/1000
6/6 - 0s - loss: 0.0019 - accuracy: 1.0000 - val_loss: 4.2377 - val_accuracy: 0.4737 - 167ms/epoch - 28ms/step
Epoch 68/1000
6/6 - 0s - loss: 0.0018 - accuracy: 1.0000 - val_loss: 4.2449 - val_accuracy: 0.4737 - 146ms/epoch - 24ms/step
Epoch 69/1000
6/6 - 0s - loss: 0.0018 - accuracy: 1.0000 - val_loss: 4.2621 - val_accuracy: 0.4737 - 196ms/epoch - 33ms/step
Epoch 70/1000
6/6 - 0s - loss: 0.0017 - accuracy: 1.0000 - val_loss: 4.2740 - val_accuracy: 0.4737 - 197ms/epoch - 33ms/step
Epoch 71/1000
6/6 - 0s - loss: 0.0017 - accuracy: 1.0000 - val_loss: 4.2773 - val_accuracy: 0.4737 - 176ms/epoch - 29ms/step
Epoch 72/1000
6/6 - 0s - loss: 0.0017 - accuracy: 1.0000 - val_loss: 4.2858 - val_accuracy: 0.4737 - 160ms/epoch - 27ms/step
Epoch 73/1000
6/6 - 0s - loss: 0.0016 - accuracy: 1.0000 - val_loss: 4.2979 - val_accuracy: 0.4737 - 150ms/epoch - 25ms/step


Epoch 130/1000
6/6 - 0s - loss: 5.3616e-04 - accuracy: 1.0000 - val_loss: 4.7291 - val_accuracy: 0.4737 - 135ms/epoch - 22ms/step
Epoch 131/1000
6/6 - 0s - loss: 5.2375e-04 - accuracy: 1.0000 - val_loss: 4.7352 - val_accuracy: 0.4737 - 131ms/epoch - 22ms/step
Epoch 132/1000
6/6 - 0s - loss: 5.1431e-04 - accuracy: 1.0000 - val_loss: 4.7431 - val_accuracy: 0.4737 - 125ms/epoch - 21ms/step
Epoch 133/1000
6/6 - 0s - loss: 5.0577e-04 - accuracy: 1.0000 - val_loss: 4.7499 - val_accuracy: 0.4737 - 127ms/epoch - 21ms/step
Epoch 134/1000
6/6 - 0s - loss: 4.9676e-04 - accuracy: 1.0000 - val_loss: 4.7568 - val_accuracy: 0.4737 - 130ms/epoch - 22ms/step
Epoch 135/1000
6/6 - 0s - loss: 4.8701e-04 - accuracy: 1.0000 - val_loss: 4.7636 - val_accuracy: 0.4737 - 132ms/epoch - 22ms/step
Epoch 136/1000
6/6 - 0s - loss: 4.7972e-04 - accuracy: 1.0000 - val_loss: 4.7654 - val_accuracy: 0.4737 - 130ms/epoch - 22ms/step
Epoch 137/1000
6/6 - 0s - loss: 4.7182e-04 - accuracy: 1.0000 - val_loss: 4.7689 - val_acc

Epoch 193/1000
6/6 - 0s - loss: 2.2621e-04 - accuracy: 1.0000 - val_loss: 5.0617 - val_accuracy: 0.4737 - 134ms/epoch - 22ms/step
Epoch 194/1000

Epoch 194: saving model to training_2\1.ckpt
6/6 - 0s - loss: 2.2345e-04 - accuracy: 1.0000 - val_loss: 5.0663 - val_accuracy: 0.4737 - 187ms/epoch - 31ms/step
Epoch 195/1000
6/6 - 0s - loss: 2.2121e-04 - accuracy: 1.0000 - val_loss: 5.0725 - val_accuracy: 0.4737 - 130ms/epoch - 22ms/step
Epoch 196/1000
6/6 - 0s - loss: 2.1872e-04 - accuracy: 1.0000 - val_loss: 5.0752 - val_accuracy: 0.4737 - 141ms/epoch - 23ms/step
Epoch 197/1000
6/6 - 0s - loss: 2.1651e-04 - accuracy: 1.0000 - val_loss: 5.0772 - val_accuracy: 0.4737 - 134ms/epoch - 22ms/step
Epoch 198/1000
6/6 - 0s - loss: 2.1446e-04 - accuracy: 1.0000 - val_loss: 5.0814 - val_accuracy: 0.4737 - 131ms/epoch - 22ms/step
Epoch 199/1000
6/6 - 0s - loss: 2.1205e-04 - accuracy: 1.0000 - val_loss: 5.0860 - val_accuracy: 0.4737 - 139ms/epoch - 23ms/step
Epoch 200/1000
6/6 - 0s - loss: 2.0971e-04 -

Epoch 255/1000
6/6 - 0s - loss: 1.2484e-04 - accuracy: 1.0000 - val_loss: 5.2917 - val_accuracy: 0.4737 - 144ms/epoch - 24ms/step
Epoch 256/1000
6/6 - 0s - loss: 1.2371e-04 - accuracy: 1.0000 - val_loss: 5.2966 - val_accuracy: 0.4737 - 146ms/epoch - 24ms/step
Epoch 257/1000
6/6 - 0s - loss: 1.2265e-04 - accuracy: 1.0000 - val_loss: 5.3000 - val_accuracy: 0.4737 - 198ms/epoch - 33ms/step
Epoch 258/1000
6/6 - 0s - loss: 1.2167e-04 - accuracy: 1.0000 - val_loss: 5.3038 - val_accuracy: 0.4737 - 186ms/epoch - 31ms/step
Epoch 259/1000
6/6 - 0s - loss: 1.2069e-04 - accuracy: 1.0000 - val_loss: 5.3067 - val_accuracy: 0.4737 - 180ms/epoch - 30ms/step
Epoch 260/1000
6/6 - 0s - loss: 1.1969e-04 - accuracy: 1.0000 - val_loss: 5.3103 - val_accuracy: 0.4737 - 163ms/epoch - 27ms/step
Epoch 261/1000
6/6 - 0s - loss: 1.1861e-04 - accuracy: 1.0000 - val_loss: 5.3139 - val_accuracy: 0.4737 - 136ms/epoch - 23ms/step
Epoch 262/1000
6/6 - 0s - loss: 1.1778e-04 - accuracy: 1.0000 - val_loss: 5.3181 - val_acc

...

Epoch 694/1000
6/6 - 0s - loss: 1.0733e-05 - accuracy: 1.0000 - val_loss: 6.2455 - val_accuracy: 0.4737 - 139ms/epoch - 23ms/step
Epoch 695/1000
6/6 - 0s - loss: 1.0696e-05 - accuracy: 1.0000 - val_loss: 6.2483 - val_accuracy: 0.4737 - 158ms/epoch - 26ms/step
Epoch 696/1000
6/6 - 0s - loss: 1.0646e-05 - accuracy: 1.0000 - val_loss: 6.2510 - val_accuracy: 0.4737 - 172ms/epoch - 29ms/step
Epoch 697/1000
6/6 - 0s - loss: 1.0610e-05 - accuracy: 1.0000 - val_loss: 6.2516 - val_accuracy: 0.4737 - 174ms/epoch - 29ms/step
Epoch 698/1000
6/6 - 0s - loss: 1.0564e-05 - accuracy: 1.0000 - val_loss: 6.2531 - val_accuracy: 0.4737 - 133ms/epoch - 22ms/step
Epoch 699/1000
6/6 - 0s - loss: 1.0525e-05 - accuracy: 1.0000 - val_loss: 6.2554 - val_accuracy: 0.4737 - 133ms/epoch - 22ms/step
Epoch 700/1000

Epoch 700: saving model to training_2\1.ckpt
6/6 - 0s - loss: 1.0464e-05 - accuracy: 1.0000 - val_loss: 6.2568 - val_accuracy: 0.4737 - 185ms/epoch - 31ms/step
Epoch 701/1000
6/6 - 0s - loss: 1.0430e-05 -

Epoch 756/1000
6/6 - 0s - loss: 8.2926e-06 - accuracy: 1.0000 - val_loss: 6.3459 - val_accuracy: 0.4737 - 162ms/epoch - 27ms/step
Epoch 757/1000
6/6 - 0s - loss: 8.2557e-06 - accuracy: 1.0000 - val_loss: 6.3476 - val_accuracy: 0.4737 - 137ms/epoch - 23ms/step
Epoch 758/1000
6/6 - 0s - loss: 8.2167e-06 - accuracy: 1.0000 - val_loss: 6.3493 - val_accuracy: 0.4737 - 184ms/epoch - 31ms/step
Epoch 759/1000
6/6 - 0s - loss: 8.1842e-06 - accuracy: 1.0000 - val_loss: 6.3503 - val_accuracy: 0.4737 - 147ms/epoch - 24ms/step
Epoch 760/1000
6/6 - 0s - loss: 8.1474e-06 - accuracy: 1.0000 - val_loss: 6.3524 - val_accuracy: 0.4737 - 135ms/epoch - 22ms/step
Epoch 761/1000
6/6 - 0s - loss: 8.1257e-06 - accuracy: 1.0000 - val_loss: 6.3539 - val_accuracy: 0.4737 - 149ms/epoch - 25ms/step
Epoch 762/1000
6/6 - 0s - loss: 8.0888e-06 - accuracy: 1.0000 - val_loss: 6.3554 - val_accuracy: 0.4737 - 153ms/epoch - 25ms/step
Epoch 763/1000
6/6 - 0s - loss: 8.0650e-06 - accuracy: 1.0000 - val_loss: 6.3576 - val_acc

Epoch 819/1000
6/6 - 0s - loss: 6.4221e-06 - accuracy: 1.0000 - val_loss: 6.4447 - val_accuracy: 0.4737 - 190ms/epoch - 32ms/step
Epoch 820/1000
6/6 - 0s - loss: 6.3918e-06 - accuracy: 1.0000 - val_loss: 6.4458 - val_accuracy: 0.4737 - 180ms/epoch - 30ms/step
Epoch 821/1000
6/6 - 0s - loss: 6.3722e-06 - accuracy: 1.0000 - val_loss: 6.4469 - val_accuracy: 0.4737 - 153ms/epoch - 25ms/step
Epoch 822/1000
6/6 - 0s - loss: 6.3549e-06 - accuracy: 1.0000 - val_loss: 6.4485 - val_accuracy: 0.4737 - 148ms/epoch - 25ms/step
Epoch 823/1000
6/6 - 0s - loss: 6.3181e-06 - accuracy: 1.0000 - val_loss: 6.4511 - val_accuracy: 0.4737 - 134ms/epoch - 22ms/step
Epoch 824/1000
6/6 - 0s - loss: 6.2964e-06 - accuracy: 1.0000 - val_loss: 6.4524 - val_accuracy: 0.4737 - 127ms/epoch - 21ms/step
Epoch 825/1000
6/6 - 0s - loss: 6.2682e-06 - accuracy: 1.0000 - val_loss: 6.4543 - val_accuracy: 0.4737 - 142ms/epoch - 24ms/step
Epoch 826/1000
6/6 - 0s - loss: 6.2509e-06 - accuracy: 1.0000 - val_loss: 6.4565 - val_acc

Epoch 882/1000
6/6 - 0s - loss: 5.0241e-06 - accuracy: 1.0000 - val_loss: 6.5398 - val_accuracy: 0.4737 - 133ms/epoch - 22ms/step
Epoch 883/1000
6/6 - 0s - loss: 4.9981e-06 - accuracy: 1.0000 - val_loss: 6.5419 - val_accuracy: 0.4737 - 188ms/epoch - 31ms/step
Epoch 884/1000
6/6 - 0s - loss: 4.9808e-06 - accuracy: 1.0000 - val_loss: 6.5436 - val_accuracy: 0.4737 - 145ms/epoch - 24ms/step
Epoch 885/1000
6/6 - 0s - loss: 4.9613e-06 - accuracy: 1.0000 - val_loss: 6.5429 - val_accuracy: 0.4737 - 143ms/epoch - 24ms/step
Epoch 886/1000
6/6 - 0s - loss: 4.9504e-06 - accuracy: 1.0000 - val_loss: 6.5449 - val_accuracy: 0.4737 - 138ms/epoch - 23ms/step
Epoch 887/1000

Epoch 887: saving model to training_2\1.ckpt
6/6 - 0s - loss: 4.9266e-06 - accuracy: 1.0000 - val_loss: 6.5470 - val_accuracy: 0.4737 - 188ms/epoch - 31ms/step
Epoch 888/1000
6/6 - 0s - loss: 4.9027e-06 - accuracy: 1.0000 - val_loss: 6.5490 - val_accuracy: 0.4737 - 119ms/epoch - 20ms/step
Epoch 889/1000
6/6 - 0s - loss: 4.8897e-06 -

Epoch 944/1000
6/6 - 0s - loss: 3.9556e-06 - accuracy: 1.0000 - val_loss: 6.6299 - val_accuracy: 0.4737 - 142ms/epoch - 24ms/step
Epoch 945/1000
6/6 - 0s - loss: 3.9426e-06 - accuracy: 1.0000 - val_loss: 6.6307 - val_accuracy: 0.4737 - 162ms/epoch - 27ms/step
Epoch 946/1000
6/6 - 0s - loss: 3.9166e-06 - accuracy: 1.0000 - val_loss: 6.6322 - val_accuracy: 0.4737 - 151ms/epoch - 25ms/step
Epoch 947/1000
6/6 - 0s - loss: 3.9057e-06 - accuracy: 1.0000 - val_loss: 6.6332 - val_accuracy: 0.4737 - 161ms/epoch - 27ms/step
Epoch 948/1000
6/6 - 0s - loss: 3.8905e-06 - accuracy: 1.0000 - val_loss: 6.6343 - val_accuracy: 0.4737 - 163ms/epoch - 27ms/step
Epoch 949/1000
6/6 - 0s - loss: 3.8884e-06 - accuracy: 1.0000 - val_loss: 6.6359 - val_accuracy: 0.4737 - 163ms/epoch - 27ms/step
Epoch 950/1000
6/6 - 0s - loss: 3.8775e-06 - accuracy: 1.0000 - val_loss: 6.6373 - val_accuracy: 0.4737 - 158ms/epoch - 26ms/step
Epoch 951/1000
6/6 - 0s - loss: 3.8580e-06 - accuracy: 1.0000 - val_loss: 6.6397 - val_acc

In [74]:
textcnnmodel.save('./tmp/modellatest')

INFO:tensorflow:Assets written to: ./tmp/modellatest\assets


INFO:tensorflow:Assets written to: ./tmp/modellatest\assets


In [75]:
newmodel = create_model()
modelpath='./tmp/modellatest'
newmodel = tf.keras.models.load_model(modelpath)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 200)          31000     
                                                                 
 conv1d_3 (Conv1D)           (None, 92, 128)           230528    
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 10)                1290      
                                                                 
 dense_7 (Dense)             (None, 10)                110       
                                                                 
Total params: 262,928
Trainable params: 262,928
Non-trainable params: 0
________________________________________________

In [12]:
# newmodel.save_weights('./model2')

In [128]:
newT = input("lol: ")

lol: Why does my dad hate me


In [134]:
from tensorflow import keras
pad_sequences=keras.preprocessing.sequence.pad_sequences
textsample=[newT]
tokenizer.fit_on_texts(textsample)

Xcnn_sample = tokenizer.texts_to_sequences(textsample)
maxlen = 100
Xcnn_sample = pad_sequences(Xcnn_sample, padding='post', maxlen=maxlen)

prediction=newmodel.predict(np.array(Xcnn_sample))


print(prediction)

print(textsample[0])
print(Xcnn_sample[0]) 

print(len(prediction[0]))
deptype=["Natural Satement","Study Problems","Peers Problems","Relationship Problems","Loneliness","Hatered","Work Problems","Bored","Unmotivated","Financial Problems"]
indeks=int(np.where(prediction[0]==np.amax(prediction[0]))[0])
print(deptype[indeks])

1/1 [==============================] - 0s 25ms/step
[[2.4541663e-03 2.3443840e-04 5.2877683e-02 2.3614230e-05 3.7681291e-04
  3.4625508e-12 9.4403255e-01 6.3504876e-07 9.7595588e-08 3.0186669e-09]]
Why does my dad hate me
[ 4 19  2 27  3  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]
10
Work Problems


In [132]:
# newmodel.save('./tmp/modellatest')